In [ ]:
# importing necessary libraries
# pandas for etl, openpyxl for excel export, nsepython for data fetch from NSE
import pandas as pd
from datetime import date
from nsepython import *
import numpy as np

In [ ]:
# assigning the dates for which application will run
end_date = (date.today() + datetime.timedelta(days=-1))
start_date = (date.today() + datetime.timedelta(days=-300))
delta = end_date - start_date   # returns timedelta

In [ ]:
# get_bhavcopy is the function to fetch delivery and total quantity
# https://unofficed.com/nse-python/documentation/nsepy/ ------official documentation

# Create an empty DataFrame
df_final = pd.DataFrame()

for i in range(delta.days + 1):
    day = start_date + datetime.timedelta(days=i)
    try:
        bhav_df = get_bhavcopy(day.strftime('%d-%m-%Y'))
        bhav_df = bhav_df.rename(columns=lambda x: x.strip())
        bhav_df['SERIES'] = bhav_df['SERIES'].str.strip()
        df_final = pd.concat([df_final, bhav_df], ignore_index=True)
        df_final = df_final[df_final['SERIES'] == 'EQ'] #filtering for 'EQ' as other serieses also coming
    except:
        print("This date is a holiday or weekend: ", day.strftime('%d-%m-%Y'))
        continue

In [20]:
# master data for getting full name per symbol
symbol_master = pd.read_excel('Stock_Symbol_Master_Data.xlsx')
# symbol_master

# creating an empty dataframe to store final results
final_df = pd.DataFrame()

In [21]:
for index, row in symbol_master.iterrows():
    print(row['SYMBOL'])

    combined_df = df_final[(df_final['SYMBOL'] == row['SYMBOL'])]
    combined_df = combined_df.drop_duplicates()
    # Convert 'Date' column from string to date
    combined_df['DATE1'] = pd.to_datetime(combined_df['DATE1'].str.strip(), format='%d-%b-%Y')
    combined_df = combined_df.sort_values(by=['SYMBOL','DATE1'])

    combined_df = combined_df[['DATE1','SYMBOL','TTL_TRD_QNTY','NO_OF_TRADES','DELIV_QTY','DELIV_PER','CLOSE_PRICE']]
    combined_df[["DELIV_QTY", "DELIV_PER"]] = combined_df[["DELIV_QTY", "DELIV_PER"]].apply(pd.to_numeric)
    combined_df['change'] = combined_df['CLOSE_PRICE'].diff()

    mask = combined_df.SYMBOL != combined_df.SYMBOL.shift(1)
    combined_df['change'][mask] = np.nan

    combined_df['gain'] = combined_df.change.mask(combined_df.change < 0, 0.0)
    combined_df['loss'] = -combined_df.change.mask(combined_df.change > 0, -0.0)
    def rma(x, n):
        """Running moving average"""
        a = np.full_like(x, np.nan)
        a[n] = x[1:n+1].mean()
        for i in range(n+1, len(x)):
            a[i] = (a[i-1] * (n - 1) + x[i]) / n
        return a

    combined_df['avg_gain'] = rma(combined_df.gain.to_numpy(), 14)
    combined_df['avg_loss'] = rma(combined_df.loss.to_numpy(), 14)

    combined_df['rs'] = combined_df.avg_gain / combined_df.avg_loss
    combined_df['rsi'] = 100 - (100 / (1 + combined_df.rs))

    final_df = pd.concat([final_df,combined_df], ignore_index=True)
    print(len(final_df))

20MICRONS
151
360ONE
355
3IINFOLTD


C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  combined_df['change'][mask] = np.nan
C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: SettingWithCopyWarning

559
3MINDIA
763
5PAISA
967
AARON


C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  combined_df['change'][mask] = np.nan
C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: SettingWithCopyWarning

1147
AARTECH
1233
AARTIDRUGS
1437
AARTIIND
1641
AARTIPHARM


...

C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  combined_df['change'][mask] = np.nan
C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: SettingWithCopyWarning

303804
ZODIACLOTH
304002
ZOMATO
304206
ZOTA
304410
ZUARI
304614
ZUARIIND


C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  combined_df['change'][mask] = np.nan
C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: SettingWithCopyWarning

304818
ZYDUSLIFE
305022
ZYDUSWELL
305226


C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  combined_df['change'][mask] = np.nan
C:\Users\Krish\AppData\Local\Temp\ipykernel_14528\3349612712.py:15: SettingWithCopyWarning

In [23]:
final_df = final_df.rename(columns={'DATE1': 'DATE', 'TTL_TRD_QNTY': 'TOTAL_TRADE_QTY', 'DELIV_QTY': 'DELIVERY_QTY',
                                    'DELIV_PER': 'DELIVERY_PERC', 'change': 'CHANGE', 'gain': 'GAIN',
                                    'loss': 'LOSS', 'avg_gain': 'AVG_GAIN', 'avg_loss': 'AVG_LOSS',
                                    'rs': 'RS', 'rsi': 'RSI'})


In [25]:
file_name = 'StockMarketData.xlsx'
final_df.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')